Найдите любой другой сервис, подходящий для развертывания моделей и реализуйте произвольную задачу (разбор данного задания будет осуществлен на сервисе https://anvil.works/ для задачи классификации базы Ирисов)

P.s. вы можете взять любой другой сервис и любую другую задачу

In [ ]:
!pip install anvil-uplink

     |████████████████████████████████| 64 kB 2.2 MB/s 
     |████████████████████████████████| 51 kB 208 kB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45229 sha256=6f0b547de5e24af287451e2ebf21dcf4790a8ac75a964146ab0321329423d796
  Stored in directory: /root/.cache/pip/wheels/29/ea/7d/3410aa0aa0e4402ead9a7a97ab2214804887e0f5c2b76f0c96
Successfully built ws4py


In [ ]:
import anvil.server

anvil.server.connect("server_VBPBM537EJAT47OEGZV66ESB-C7Y5PZSKJINU3ADV")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [ ]:
# Имя файла с весами модели
MODEL_NAME =   'model_air.h5'

In [ ]:
import gdown

# Загрузка файла модели из облака
gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l15/model_air.h5', None, quiet=True)

'model_air.h5'

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
from PIL import Image 
from matplotlib import cm
import io

MODEL_NAME = 'model_air.h5'
model = load_model(MODEL_NAME)                                              # Загрузка весов модели
INPUT_SHAPE = (256, 456, 3)

In [ ]:
# Загрузка тестовых изображений 10 классов
import gdown
gdown.download('https://storage.yandexcloud.net/aiueducation/Content/knowledge/test_images.zip', None, quiet=True)

# Распаковка архива
!unzip -qo test_images.zip

In [ ]:
@anvil.server.callable

def original_image(image_number=1):
    # Просмотр тестового изображения
    # Путь к файлу изображения
    test_path = f'/content/test_images/{image_number}.jpg'  
    img = Image.open(test_path)
    
    img_byte_arr = io.BytesIO()
    img.save(img_byte_arr, format='JPEG')
    img_byte_arr = img_byte_arr.getvalue()
    media_obj = anvil.BlobMedia(content_type="image/jpeg", content=img_byte_arr)
   
    return media_obj

In [ ]:
@anvil.server.callable

def process(image_number=1):
    test_path = f'/content/test_images/{image_number}.jpg'  
    image = Image.open(test_path)
    resized_image = image.resize((INPUT_SHAPE[1], INPUT_SHAPE[0]))          # Изменение размера изображения в соответствии со входом сети
    array = np.array(resized_image)[..., :3][np.newaxis, ..., np.newaxis]   # Регулировка формы тензора для подачи в сеть
    prediction_array = (255 * model.predict(array)).astype(int)             # Запуск предсказания сети
    prediction_array = np.split(prediction_array, 2, axis = -1)[0]          # Нулевой канал предсказания (значения 0 - самолет, 1 - фон)
    zeros = np.zeros_like(prediction_array)                                 # Создание массива нулей
    ones = np.ones_like(prediction_array)                                   # Создание массива единиц
    prediction_array_4d = np.concatenate([255 * (prediction_array > 100), zeros, zeros, 128 * ones], axis=3)[0].astype(np.uint8)  # Формирование тензора для наложения найденной маски
    mask_image = Image.fromarray(prediction_array_4d).resize(image.size)    # Преобразование тензора в изображение и подгонка его размера к исходному
    image.paste(mask_image, (0, 0), mask_image)                             # Добавление маски на исходное изображение

    #convert img to anvil's data format
    img_byte_arr = io.BytesIO()
    resized_image.save(img_byte_arr, format='JPEG')
    img_byte_arr = img_byte_arr.getvalue()
    resized_image_anvil = anvil.BlobMedia(content_type="image/jpeg", content=img_byte_arr)

    #convert numpy array to image
    prediction_array = Image.fromarray(np.uint8(cm.gist_earth(prediction_array[0].squeeze(axis=2))*255))
    prediction_array = prediction_array.convert('RGB')
    #convert img to anvil's data format
    img_byte_arr = io.BytesIO()
    prediction_array.save(img_byte_arr, format='JPEG')
    img_byte_arr = img_byte_arr.getvalue()
    prediction_array_anvil = anvil.BlobMedia(content_type="image/jpeg", content=img_byte_arr)
    
    #convert img to anvil's data format
    img_byte_arr = io.BytesIO()
    image.save(img_byte_arr, format='JPEG')
    img_byte_arr = img_byte_arr.getvalue()
    image_anvil = anvil.BlobMedia(content_type="image/jpeg", content=img_byte_arr)

    return resized_image_anvil, prediction_array_anvil, image_anvil